# Precomputing values for use in fits of Stan models

Because of the way Stan works, it is necessary to compute some values in advance which can then be passed into the fit an interpolated over. The precomputed values will be different for different sets of source distances, and therefore different catalogues. 

Here we show how to compute the values for the SBG catalogue, but it is exactly the same for all cases, just changing the input label.

For ease, all the precomputed table files used are provided for use in this repository.

In [1]:
import os
import h5py

from fancy import Data, Model, Analysis
import numpy as np

# detector_type = "PAO"
detector_type = "TA"

In [2]:
'''set detector and detector properties'''
if detector_type == "TA":
    from fancy.detector.TA2015 import detector_properties, Eth
elif detector_type == "PAO":
    from fancy.detector.auger2014 import detector_properties, Eth
else:
    raise Exception("Undefined detector type!")


In [3]:
# Define file containing catalogue information
source_file = '../../data/sourcedata.h5'

# Path to Stan files
stan_path = '../../stan/'

# make output directory if it doesnt exist
if not os.path.isdir("output"):
    os.mkdir("output")

# source_types = ["SBG_23", "2FHL_250Mpc"]
source_types = ["SBG_23"]

# exposure factor
# exp_factors = [1., 5., 10., 100.]
exp_factors = [1., 5.]

# store original total exposure and area to restore per iteration
alpha_T_0 = detector_properties["alpha_T"]
A_0 = detector_properties["A"]

for source_type in source_types:
    for exp_factor in exp_factors:
        print("Current Source: {0}".format(source_type))
        print("Current Exposure Factor: {0}".format(exp_factor))

        # File in which to store precomputation
        table_file = "output/tables_{0}_{2}_epsx{1:.0f}.h5".format(source_type, exp_factor, detector_type)

        # modify exposure factor
        detector_properties["alpha_T"] *= exp_factor
        detector_properties["A"] *= exp_factor

        print("alpha_T is properly configured: ", detector_properties["alpha_T"] / exp_factor == alpha_T_0)
        print("A is properly configured: ", detector_properties["A"] / exp_factor == A_0)

        data = Data()
        data.add_source(source_file, source_type) 
        data.add_detector(detector_properties)  # KW: add detector information

        model_name = 'joint_model.stan'
        model = Model(model_filename = model_name, include_paths = stan_path)
        model.input(Eth = Eth) # EeV

        summary = b'Precomputation for Exposure and Energies'
        analysis = Analysis(data, model, analysis_type = 'joint', 
                            filename = table_file, summary = summary)

        analysis.build_tables(fit_only = True)
        analysis.tables.save(table_file)

        analysis.build_energy_table(table_file = table_file)

        # reset exposure factor to original
        detector_properties["alpha_T"] = alpha_T_0
        detector_properties["A"] = A_0

Current Source: SBG_23
Current Exposure Factor: 0.5
True
True


Precomputing exposure integral: 100%|██████████| 23/23 [03:28<00:00,  9.09s/it]


Precomputing exposure integral: 100%|██████████| 23/23 [00:51<00:00,  2.25s/it]
